# Crime Statistics Resampling Techniques

### Import Dependencies

In [1]:
import os
import warnings
warnings.filterwarnings('ignore')

In [2]:
import numpy as np
import pandas as pd
from pathlib import Path
from collections import Counter
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

### Read the CSV and Perform Basic Data Cleaning

In [3]:
# Load the data
df = pd.read_csv('../Resources/CSV/crime_data.csv')

In [4]:
# Drop the year column to not skew data with year count and averages
df = df.drop(columns =['Year'])
df.head()

,County,Agency_Count,Murder,Rape,Assault,Burglary,Larceny,Auto_Theft,Violent_Offenses,NonViolent_Offenses,Total_Crime,Population
0,Austin County,5,0,8,53,121,283,13,61,417,482,29354
1,Austin County,5,0,6,43,136,239,31,49,406,466,29718
2,Austin County,5,0,11,34,98,183,45,45,326,376,29963
3,Austin County,5,0,11,32,82,138,29,43,249,295,29912
4,Austin County,5,0,5,38,80,147,29,43,256,302,30009


In [5]:
# Convert the target column values to violent and nonviolent based on their crime
x = dict.fromkeys(['Murder', 'Rape', 'Assault'], 'violent_crimes')    
df = df.replace(x)

x = dict.fromkeys(['Burglary', 'Larceny', 'Auto_Theft'], 'nonviolent_crimes')    
df = df.replace(x)

df.reset_index(inplace=True, drop=True)

df.head()

,County,Agency_Count,Murder,Rape,Assault,Burglary,Larceny,Auto_Theft,Violent_Offenses,NonViolent_Offenses,Total_Crime,Population
0,Austin County,5,0,8,53,121,283,13,61,417,482,29354
1,Austin County,5,0,6,43,136,239,31,49,406,466,29718
2,Austin County,5,0,11,34,98,183,45,45,326,376,29963
3,Austin County,5,0,11,32,82,138,29,43,249,295,29912
4,Austin County,5,0,5,38,80,147,29,43,256,302,30009


### Split the Data into Training and Testing

In [6]:
# Create dummies
df = pd.get_dummies(df.drop(columns='County'))
df.head()

,Agency_Count,Murder,Rape,Assault,Burglary,Larceny,Auto_Theft,Violent_Offenses,NonViolent_Offenses,Total_Crime,Population
0,5,0,8,53,121,283,13,61,417,482,29354
1,5,0,6,43,136,239,31,49,406,466,29718
2,5,0,11,34,98,183,45,45,326,376,29963
3,5,0,11,32,82,138,29,43,249,295,29912
4,5,0,5,38,80,147,29,43,256,302,30009


In [7]:
# Create target
y = pd.DataFrame(df["Violent_Offenses"])

# Create features
X = pd.DataFrame(df[['Population', 'Agency_Count','Total_Crime']])

In [8]:
X.describe()

,Population,Agency_Count,Total_Crime
count,1.020000e+02,102.000000,102.000000
mean,4.378851e+05,8.617647,14348.470588
std,1.093114e+06,10.573687,45215.295287
min,2.530600e+04,1.000000,281.000000
25%,3.543850e+04,3.000000,820.750000
50%,5.080850e+04,4.000000,1166.000000
75%,3.503322e+05,11.000000,6549.500000
max,4.799254e+06,45.000000,197686.000000


In [9]:
y.head()

,Violent_Offenses
0,61
1,49
2,45
3,43
4,43


In [10]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
X_train.shape

(76, 3)

## Oversampling

### Naive Random Oversampling

In [11]:
# Resample the training data with the RandomOversampler
from imblearn.over_sampling import RandomOverSampler

ros = RandomOverSampler(random_state=1)

# resampling
X_resampled, y_resampled = ros.fit_resample(X_train, y_train)

In [12]:
# Resample the training data with the RandomOversampler
Counter(y_resampled)

Counter({'Violent_Offenses': 1})

In [13]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_resampled, y_resampled)

LogisticRegression(random_state=1)

In [14]:
# Create Model Prediction
y_pred = model.predict(X_test)

In [15]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

0.0

In [16]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = model.predict(X_test)
confusion_matrix(y_test, y_pred)

array([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 1],
       [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 2],
       [0, 0, 0, 0, 

In [17]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

         33       0.00      0.00      1.00      0.00      0.00      0.00       1.0
         39       0.00      0.00      1.00      0.00      0.00      0.00       1.0
         48       0.00      0.00      1.00      0.00      0.00      0.00       1.0
         85       0.00      0.00      1.00      0.00      0.00      0.00       1.0
         86       0.00      0.00      1.00      0.00      0.00      0.00       1.0
        104       0.00      0.00      1.00      0.00      0.00      0.00       1.0
        117       0.00      0.00      1.00      0.00      0.00      0.00       1.0
        129       0.00      0.00      1.00      0.00      0.00      0.00       1.0
        130       0.00      0.00      1.00      0.00      0.00      0.00       1.0
        135       0.00      0.00      1.00      0.00      0.00      0.00       2.0
        143       0.00      0.00      1.00      0.00      0.00      0.00       1.0
   

### SMOTE Oversampling

In [18]:
# Resample the training data with SMOTE
from imblearn.over_sampling import SMOTE

X_resampled, y_resampled = SMOTE(random_state=1, sampling_strategy='auto').fit_resample(X_train, y_train)

Counter(y_resampled)

ValueError: Expected n_neighbors <= n_samples,  but n_samples = 1, n_neighbors = 6

In [ ]:
# Train the Logistic Regression model using the resampled data
model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_resampled, y_resampled)

In [ ]:
# Create Model Prediction
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
print(classification_report_imbalanced(y_test, y_pred))

## Undersampling

In [ ]:
# Resample the data using the ClusterCentroids resampler
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.under_sampling import ClusterCentroids

cc = ClusterCentroids(random_state=1)

X_resampled, y_resampled = cc.fit_resample(X_train, y_train)

Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=78)

model.fit(X_resampled, y_resampled)

In [ ]:
# Create Model Prediction
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))

## Combination (Over and Under) Sampling

In [ ]:
# Resample the training data with SMOTEENN
# Warning: This is a large dataset, and this step may take some time to complete
from imblearn.combine import SMOTEENN

smote_enn = SMOTEENN(random_state=0)

X_resampled, y_resampled = smote_enn.fit_resample(X, y)

Counter(y_resampled)

In [ ]:
# Train the Logistic Regression model using the resampled data
from sklearn.linear_model import LogisticRegression

model = LogisticRegression(solver='lbfgs', random_state=1)

model.fit(X_resampled, y_resampled)

In [ ]:
# Create Model Prediction
y_pred = model.predict(X_test)

In [ ]:
# Calculated the balanced accuracy score
from sklearn.metrics import balanced_accuracy_score

balanced_accuracy_score(y_test, y_pred)

In [ ]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(y_test, y_pred)

In [ ]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(y_test, y_pred))